## Project 4
## Submitted by Roshan Lulu

### Step 1 - Get the Job Posting data from the website

- Identify the Job titles to scrap
- Identify the element/child relationships and get the attributes
- Baby step - Get 1 job posting scraped


In [4]:
## Data Scientist
ds1_syd = 'https://au.jora.com/j?q=Data+Scientist&l=Sydney+NSW&sp=homepage'

In [16]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import scrapy
from scrapy import Selector

In [19]:
def parse_url(url):
    
    response  =  requests.get(url)
    links     =  Selector(text=response.text).xpath("//td[@class='  row  result']/text()").extract()
    print (links)
        
df = parse_url(ds1_syd)
df

[]
